<a href="https://colab.research.google.com/github/KiyokazuTakeuchi/my-first-repo/blob/master/theaterhouse123.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import matplotlib.pyplot as plt

# アクセスログ(transactionRevenue)の読み込み  
  

```
ga:dimension9,ga:dateHourMinute,ga:channelGrouping,ga:sessions,ga:avgSessionDuration,ga:bounceRate,ga:goal3Completions,ga:goal4Completions,	ga:goal5Completions,ga:transactionRevenue
```



In [3]:
# ga:transactionRevenueを含むファイル(このファイルの情報を元にする)
input = '/content/drive/Shareddrives/FPO-未来企画室/ＡＩラボ関連/シアターハウス/アナリティクス分析/ミニ合宿/o_20200801-20210531.csv'
# input = '/content/drive/Shareddrives/FPO-未来企画室/ＡＩラボ関連/シアターハウス/アナリティクス分析/ミニ合宿/o_202103-022105.csv'

In [4]:
df = pd.read_csv(input)
df.drop('Unnamed: 0', axis = 1, inplace=True)

In [5]:
# sz = df.shape[0] // 2
# df_bottom = df[sz:]
# df_bottom.to_csv('/content/drive/Shareddrives/FPO-未来企画室/ＡＩラボ関連/シアターハウス/アナリティクス分析/ミニ合宿/o_202103-022105.csv')

In [6]:
# 主要な項目にデータ型を設定する
df['ga:dimension9'] = df['ga:dimension9'].astype(str)
df['ga:dateHourMinute'] = df['ga:dateHourMinute'].astype(str)
df['datetime'] = pd.to_datetime(df['ga:dateHourMinute'])

In [7]:
# ga:channelGroupingをOne-hot encoding後、ga:dimension9で集約
df = pd.get_dummies(df, columns=['ga:channelGrouping'])
df_all = df.groupby(['ga:dimension9'], as_index=False).sum()

In [8]:
df_all.to_csv('/content/drive/Shareddrives/FPO-未来企画室/ＡＩラボ関連/シアターハウス/アナリティクス分析/ミニ合宿/all_revenue.csv', index=False)

# アクセスログ(パス)の読み込み　　


```
ga:dimension9,ga:dateHourMinute,ga:pagePath,ga:sessions,ga:avgSessionDuration,
```



In [9]:
input2 = '/content/drive/Shareddrives/FPO-未来企画室/ＡＩラボ関連/シアターハウス/アナリティクス分析/ミニ合宿/閲覧しているページ比較_20200801-20210531.csv'
df2 = pd.read_csv(input2)
df2.drop('Unnamed: 0', axis = 1, inplace=True)
# 念のため主要な項目にデータ型を設定する
df2['ga:dimension9'] = df2['ga:dimension9'].astype(str)
df2['ga:dateHourMinute'] = df2['ga:dateHourMinute'].astype(str)
df2['datetime'] = pd.to_datetime(df2['ga:dateHourMinute'])

# ga:sessions/ga:avgSessionDurationは不要(もう一方のファイルに存在)のため削除する
df2.drop('ga:sessions', axis=1, inplace=True)
df2.drop('ga:avgSessionDuration', axis=1, inplace=True)

In [10]:
# pagePathからオプションデータ(?)を削除
import os
plist = [os.path.dirname(df2.loc[i, 'ga:pagePath'].split('?')[0]) for i in range(df2.shape[0])]
df2['basedir'] = pd.DataFrame(plist)
df2.drop('ga:pagePath', axis=1, inplace=True)

In [11]:
# basedirをOne-hot encording後、ga:dimension9で集約
df2 = pd.get_dummies(df2, columns=['basedir'])
df2_all = df2.groupby(['ga:dimension9'], as_index=False).sum()

In [12]:
df2_all.to_csv('/content/drive/Shareddrives/FPO-未来企画室/ＡＩラボ関連/シアターハウス/アナリティクス分析/ミニ合宿/pagePath.csv', index=False)

# ファイルをマージ(transactionRevenue+pagePath)  


In [13]:
# df1とdf2をマージする(idが一致しないデータはNaNとなる)
df_total = pd.merge(df_all, df2_all, on='ga:dimension9', how='outer')

In [14]:
df_total.to_csv('/content/drive/Shareddrives/FPO-未来企画室/ＡＩラボ関連/シアターハウス/アナリティクス分析/ミニ合宿/df_total.csv', index=False)

In [15]:
df_total_inner = pd.merge(df_all, df2_all, on='ga:dimension9', how='inner')
df_total_inner.to_csv('/content/drive/Shareddrives/FPO-未来企画室/ＡＩラボ関連/シアターハウス/アナリティクス分析/ミニ合宿/df_total_inner.csv', index=False)

# 購入者のIDを取得

In [16]:
#@title
df_rev = df[df['ga:transactionRevenue'] > 0]
df_customers = df_rev['ga:dimension9'].unique()

In [17]:
from datetime import date,timedelta

# 購入日から+1w, +2w, +1M, +2M以内のデータ件数を返す
def count_date(row):
  refd = row['datetime'] #                                     
  dt = df[df['ga:dimension9'] == row['ga:dimension9']]  # TODO: 複数回購入がある場合は不正となる可能性あり
  d1w = refd - timedelta(days=7)  # 購入日より1週間前
  d2w = refd - timedelta(days=14) #           2週間前
  d1m = refd - timedelta(days=30) #　　　　 　1ヵ月前
  d2m = refd - timedelta(days=60) #　　　　 　2ヵ月前
  return [row['ga:dimension9'], refd, len(dt[(dt['datetime'] >= d1w) & (dt['datetime'] < refd)]), len(dt[(dt['datetime'] >= d2w) & (dt['datetime'] < d1w)]), len(dt[(dt['datetime'] >= d1m) & (dt['datetime'] < d2w)]), len(dt[(dt['datetime'] >= d2m) & (dt['datetime'] < d1m)])]

cds = [count_date(row) for index, row in df_rev.iterrows()]

df_cds = pd.DataFrame(cds,  columns=['ga:dimension9', 'datetime', '7d', '14d', '1M', '2M'])

In [18]:
df_rev_cds = pd.merge(df_cds, df_rev, on=['ga:dimension9', 'datetime'])

In [19]:
df_rev_cds.to_csv('/content/drive/Shareddrives/FPO-未来企画室/ＡＩラボ関連/シアターハウス/アナリティクス分析/ミニ合宿/df_rev_cds.csv', index=False)

In [20]:
df_no_rev = df[df['ga:transactionRevenue'] == 0]

In [21]:
# 未購入データから購入者のデータを削除
for cs in df_customers:
  drop_index = df_no_rev.index[df_no_rev['ga:dimension9'] == cs] 
  df_no_rev = df_no_rev.drop(drop_index) 

In [22]:
# 未購入者データ(df_no_rev)からIDを抽出(重複なし)
df_non_customers = df_no_rev['ga:dimension9'].unique()

In [23]:
import datetime
# 購入日から+1w, +2w, +1M, +2M以内のデータ件数を返す
def count_date_norev(id, baseday):
  # refd = row['datetime'] #                                     
  dt = df_no_rev[df_no_rev['ga:dimension9'] == id]
  d1w = baseday - timedelta(days=7)  # 購入日より1週間前
  d2w = baseday - timedelta(days=14) #           2週間前
  d1m = baseday - timedelta(days=30) #　　　　 　1ヵ月前
  d2m = baseday - timedelta(days=60) #　　　　 　2ヵ月前
  return [id, len(dt[(dt['datetime'] >= d1w) & (dt['datetime'] < baseday)]), len(dt[(dt['datetime'] >= d2w) & (dt['datetime'] < d1w)]), len(dt[(dt['datetime'] >= d1m) & (dt['datetime'] < d2w)]), len(dt[(dt['datetime'] >= d2m) & (dt['datetime'] < d1m)])]

rd = datetime.datetime( 2021 , 5 , 31, 0 , 0 , 0 , 0)  # 2021/05/31 0:0:0:0
# 任意に1000間隔でIDを指定
cds_no_rev = [count_date_norev(id, rd) for id in np.nditer(df_non_customers[::1000], flags=['refs_ok'])]
# cds_no_rev = [count_date_norev(id, rd) for id in np.nditer(df_non_customers, flags=['refs_ok'])]

df_cds_no_rev = pd.DataFrame(cds_no_rev,  columns=['ga:dimension9', '7d', '14d', '1M', '2M'])

In [24]:
df_cds_no_rev.to_csv('/content/drive/Shareddrives/FPO-未来企画室/ＡＩラボ関連/シアターハウス/アナリティクス分析/ミニ合宿/df_cds_no_rev.csv', index=False)

In [25]:
df_cds_no_rev['ga:dimension9'] = df_cds_no_rev['ga:dimension9'].astype(str) #
df_no_rev_cds = pd.merge(df_cds_no_rev, df_no_rev, on=['ga:dimension9'], how='inner')

In [26]:
df_rev_no_rev = pd.concat([df_rev_cds, df_no_rev_cds], axis=0)

In [27]:
df_rev_no_rev.to_csv('/content/drive/Shareddrives/FPO-未来企画室/ＡＩラボ関連/シアターハウス/アナリティクス分析/ミニ合宿/df_rev_no_rev.csv', index=False)

# ここまで

In [ ]:
# ga:channelGroupingをOne-hot encodingする
df_one = pd.get_dummies(df, columns=['ga:channelGrouping'])

In [ ]:
# ユーザID(ga:dimension9)単位に各項目を合計
df_total = df_one.groupby(['ga:dimension9'], as_index=False).sum()

In [ ]:
# ga:pagePathを含むファイル
input2 = f'/content/drive/Shareddrives/FPO-未来企画室/ＡＩラボ関連/シアターハウス/アナリティクス分析/ミニ合宿/閲覧しているページ比較_20200801-20210531.csv'

In [ ]:
df2 = pd.read_csv(input2)
df2.drop('Unnamed: 0', axis = 1, inplace=True)

In [ ]:
# 念のため主要な項目にデータ型を指定する
df2['ga:dimension9'] = df2['ga:dimension9'].astype(str)
df2['ga:dateHourMinute'] = df2['ga:dateHourMinute'].astype(str)
df2['datetime'] = pd.to_datetime(df2['ga:dateHourMinute'])

In [ ]:
# ga:session2, ga:avgSessionDurationを削除(他方のファイルに同じデータが含まれているはず)
df2.drop('ga:sessions', axis=1, inplace=True)
df2.drop('ga:avgSessionDuration', axis=1, inplace=True)

In [ ]:
# ga:pagePathを編集(親ディレクトリのみに変換)
import os
plist = [os.path.dirname(df2.loc[i, 'ga:pagePath']) for i in range(df2.shape[0])]
df2['basedir'] = pd.DataFrame(plist)

In [ ]:
# ga:pagePath削除
df2.drop('ga:pagePath', axis=1, inplace=True)

In [ ]:
# 'basedir'をOne-hot encodingする
df2_one = pd.get_dummies(df2, columns=['basedir'])

In [ ]:
# ユーザID(ga:dimension9)単位に各項目を合計
df2_total = df2_one.groupby(['ga:dimension9'], as_index=False).sum()

In [ ]:
# df1とdf2をマージする(idが一致しないデータはNaNとなる)
df_total = pd.merge(df_total, df2_total, on='ga:dimension9', how='inner')

In [ ]:
df_total.to_csv(f'/content/drive/Shareddrives/FPO-未来企画室/ＡＩラボ関連/シアターハウス/アナリティクス分析/ミニ合宿/df_total.csv', index=False)

In [ ]:
df_sort = df.sort_values(['ga:dimension9', 'datetime'],ascending=[True, True])

In [ ]:
df2_sort = df2.sort_values(['ga:dimension9', 'datetime'],ascending=[True, True])

In [ ]:
import seaborn as sns
# 相関行列
cols = df_total.columns
cor = df_total[cols].corr()
plt.figure(figsize=(320, 320))
sns.heatmap(cor, cmap= sns.color_palette('cool', 10), annot=True,fmt='.2f', vmin = -1, vmax = 1)